THIS IS THE FINAL VERSION OF THE MODEL WHERE ALL PROCESSES WILL TAKE PLACE.

In [2]:
import numpy as np # type: ignore
import pandas as pd # type: ignore
import tensorflow as tf #type: ignore
import torch
#import sentencepiece #type:ignore
import evaluate #type:ignore
from transformers import XLNetTokenizer, XLNetModel, XLNetForSequenceClassification # type: ignore
from transformers import Trainer, TrainingArguments#type:ignore
from sklearn.preprocessing import LabelEncoder #type:ignore
from datasets import load_dataset # type: ignore



In [3]:
ds = load_dataset("Estwld/empathetic_dialogues_llm", cache_dir="./cache")
type(ds)

Generating test split: 100%|██████████| 2547/2547 [00:00<00:00, 215294.08 examples/s]


datasets.dataset_dict.DatasetDict

In [4]:
print(ds['train'][91])
print(ds['test'][100])
print(ds['valid'][100])

{'conv_id': 'hit:53_conv:107', 'situation': 'I really want to win the lottery someday, maybe i will? ', 'emotion': 'hopeful', 'conversations': [{'content': 'So this is going to sound crazy but, one day I am going to win the lottery. ', 'role': 'user'}, {'content': 'That would pretty amazing. What will you do with the winning?', 'role': 'assistant'}, {'content': 'Pay off my debts from moving number 1, and then buy a house! What would you do with the money? ', 'role': 'user'}, {'content': 'I think the same really. Making sure everything is paid off, having a roof over your head, etc... is what most people would want to take caer of first.', 'role': 'assistant'}]}
{'conv_id': 'hit:469_conv:939', 'situation': 'Whenever I hear certain songs, it brings back so many memories. Mostly of when I was young and dumb staying up late with friends.', 'emotion': 'nostalgic', 'conversations': [{'content': 'Have you ever heard a song on the radio and it just takes you back to a different time? Like an e

In [5]:
print(ds)
print(ds['train'].features)

DatasetDict({
    train: Dataset({
        features: ['conv_id', 'situation', 'emotion', 'conversations'],
        num_rows: 19533
    })
    valid: Dataset({
        features: ['conv_id', 'situation', 'emotion', 'conversations'],
        num_rows: 2770
    })
    test: Dataset({
        features: ['conv_id', 'situation', 'emotion', 'conversations'],
        num_rows: 2547
    })
})
{'conv_id': Value(dtype='string', id=None), 'situation': Value(dtype='string', id=None), 'emotion': Value(dtype='string', id=None), 'conversations': [{'content': Value(dtype='string', id=None), 'role': Value(dtype='string', id=None)}]}


In [6]:
data = ds['train']
type(data)
#print(data)
#DATA SET IS IN ARROW FORMAT BECAUSE HUGGING FACE USES IT TO EASILY TRANSFER DATA

datasets.arrow_dataset.Dataset

In [7]:
#CONVERTED THE ARROW DATASET INTO PANDAS
dftrain = ds['train'].to_pandas()
dftest = ds['test'].to_pandas()
dfvalid = ds['valid'].to_pandas()
type(dftrain)

pandas.core.frame.DataFrame

In [8]:
pd.set_option('display.max_rows', 100)  
dftrain.head(2)

,conv_id,situation,emotion,conversations
0,hit:0_conv:1,I remember going to the fireworks with my best...,sentimental,[{'content': 'I remember going to see the fire...
1,hit:1_conv:2,i used to scare for darkness,afraid,[{'content': ' it feels like hitting to blank ...


In [9]:
type(data)
print(ds['train'][1])

{'conv_id': 'hit:1_conv:2', 'situation': ' i used to scare for darkness', 'emotion': 'afraid', 'conversations': [{'content': ' it feels like hitting to blank wall when i see the darkness', 'role': 'user'}, {'content': "Oh ya? I don't really see how", 'role': 'assistant'}, {'content': 'dont you feel so.. its a wonder ', 'role': 'user'}, {'content': 'I do actually hit blank walls a lot of times but i get by', 'role': 'assistant'}, {'content': ' i virtually thought so.. and i used to get sweatings', 'role': 'user'}, {'content': 'Wait what are sweatings', 'role': 'assistant'}]}


Dataset made into a csv file


In [10]:
#dftrain.to_csv("dataset_Train.csv", index=False)
#dftest.to_csv("dataset_Test.csv", index= False)
#dfvalid.to_csv("dataset_Valid.csv", index=False)

In [11]:
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")

In [12]:
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased",num_labels = 32)

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


XLNET RAW OUTPUT

In [13]:
#independent feature:
X = dftrain['situation'].astype(str).to_list()
#dependent feature
Y = dftrain['emotion']


In [14]:
pd.get_dummies(Y,drop_first=True)

,angry,annoyed,anticipating,anxious,apprehensive,ashamed,caring,confident,content,devastated,...,joyful,lonely,nostalgic,prepared,proud,sad,sentimental,surprised,terrified,trusting
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19528,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19529,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19530,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
19531,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


In [15]:
unique_emotions = dftrain["emotion"].unique()  # List of emotions
print(unique_emotions)


['sentimental' 'afraid' 'proud' 'faithful' 'terrified' 'joyful' 'angry'
 'sad' 'jealous' 'grateful' 'prepared' 'embarrassed' 'excited' 'annoyed'
 'lonely' 'ashamed' 'guilty' 'surprised' 'nostalgic' 'confident' 'furious'
 'disappointed' 'caring' 'trusting' 'disgusted' 'anticipating' 'anxious'
 'hopeful' 'content' 'impressed' 'apprehensive' 'devastated']


In [16]:
label_encoder = LabelEncoder()
label_encoder.fit(unique_emotions)
corresponding_labels = (dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))
print(corresponding_labels)


{'afraid': np.int64(0), 'angry': np.int64(1), 'annoyed': np.int64(2), 'anticipating': np.int64(3), 'anxious': np.int64(4), 'apprehensive': np.int64(5), 'ashamed': np.int64(6), 'caring': np.int64(7), 'confident': np.int64(8), 'content': np.int64(9), 'devastated': np.int64(10), 'disappointed': np.int64(11), 'disgusted': np.int64(12), 'embarrassed': np.int64(13), 'excited': np.int64(14), 'faithful': np.int64(15), 'furious': np.int64(16), 'grateful': np.int64(17), 'guilty': np.int64(18), 'hopeful': np.int64(19), 'impressed': np.int64(20), 'jealous': np.int64(21), 'joyful': np.int64(22), 'lonely': np.int64(23), 'nostalgic': np.int64(24), 'prepared': np.int64(25), 'proud': np.int64(26), 'sad': np.int64(27), 'sentimental': np.int64(28), 'surprised': np.int64(29), 'terrified': np.int64(30), 'trusting': np.int64(31)}


In [17]:
# Move model to device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()  # Set to evaluation mode

# Function to tokenize input texts
def tokenize_texts(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors="pt").to(device)  # Convert to PyTorch tensors

# Predict function
def predict_emotion(situation):
    inputs = tokenize_texts([situation])  # Tokenize input
    with torch.no_grad():  # Disable gradient computation for inference
        outputs = model(**inputs)  # Call the model
    
    logits = outputs.logits if hasattr(outputs, "logits") else outputs[0]
    predicted_label = torch.argmax(logits, dim=1).item()  # Get class index
    return predicted_label

# Example prediction
situation_example = "I lost my job and I feel helpless."
print("Predicted Emotion (Raw Output):", predict_emotion(situation_example))


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Predicted Emotion (Raw Output): 24


In [18]:
keys = [key for key, val in corresponding_labels.items() if val == 13]
print(keys)

['embarrassed']


FINE TUNE

In [19]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
   logits, labels = eval_pred
   predictions = np.argmax(logits, axis=-1)
   return metric.compute(predictions=predictions, references=labels)

In [22]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",  # Output directory
    evaluation_strategy="epoch",  # Evaluate at each epoch
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",  # Log directory
    logging_steps=10,
    save_strategy="epoch",
)

# Initialize Trainer correctly
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dftrain,
    eval_dataset=dftest,
    tokenizer=tokenizer,  
    compute_metrics=compute_metrics,
)

C:\Users\aikka\AppData\Local\Temp\ipykernel_3172\78884559.py:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
